In [ ]:
!pip install -q transformers
!pip install -q datasets
!pip install accelerate -U
!pip install matplotlib
!pip install ipywidgets
!pip install bertviz==1.2.0
!pip install scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.2/156.2 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 81.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 9.5 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import torch
from transformers import AutoTokenizer, DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification

from datasets import load_dataset
from transformers import Trainer
from transformers import TrainingArguments
from scipy.special import factorial
import math

In [ ]:
class GaussianNaiveBayes:
    def __init__(self):
        return

    def sumlogfactorial(self,x):
      sum = 0
      for xd in x:
          sum = sum + np.log(math.factorial(xd)+1e-5)
      return sum

    def fit(self, x, y):
        N, D = x.shape
        C = np.max(y) + 1
        # one parameter for each feature conditioned on each class
        theta = np.zeros((C, D))
        Nc = np.zeros(C)  # number of instances in class c
        # for each class get the MLE for the mean and std
        for c in range(C):
            x_c = x[y == c]  # slice all the elements from class c
            Nc[c] = x_c.shape[0]  # get number of elements of class c
            theta[c, :] = x_c.sum(
                axis=0) / x_c.sum()  # counts of word d in all documents labelled c / total word count in all documents labelled c

        self.theta = theta  # C x D
        self.pi = (Nc + 1) / (
                N + C)  # Laplace smoothing (using alpha_c=1 for all c)
        return self

    def logsumexp(self, Z):  # dimension C x N
        Zmax = np.max(Z, axis=0)[None, :]  # max over C
        log_sum_exp = Zmax + np.log(np.sum(np.exp(Z - Zmax), axis=0))
        return log_sum_exp

    def predict(self, xt):
        log_prior = np.log(self.pi)[:, None]
        log_factorial = np.log(factorial(xt.sum(axis=1))) - np.apply_along_axis(self.sumlogfactorial, axis=1, arr=xt)
        # Multinomial likelihood
        log_likelihood = log_factorial + np.log(self.theta+1e-5).dot(xt.T)

        log_posterior = log_prior + log_likelihood
        posterior = np.exp(log_posterior - self.logsumexp(log_posterior)) # dimension C x N
        return np.argmax(posterior.T, axis=1)  # dimension N

    def evaluate_acc(self, y, yt):
        accuracy = np.sum(y == yt) / y.shape[0]
        return accuracy


def evaluate_acc(y, yt):
    accuracy = np.sum(y == yt) / y.shape[0]
    return accuracy

if __name__ == '__main__':
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    emotions = load_dataset("emotion")
    train = emotions['train']
    test = emotions['test']
    vectorizer = CountVectorizer()
    X_train = vectorizer.fit_transform(train['text']).toarray()
    gnb = GaussianNaiveBayes()
    gnb.fit(X_train, np.asarray(train['label']))

    yt_train = gnb.predict(X_train)
    print("train accuracy: " + str(gnb.evaluate_acc(np.asarray(train['label']), yt_train)))

    X_test = vectorizer.transform(test['text']).toarray()
    yt_test = gnb.predict(X_test)
    print("test accuracy: " + str(gnb.evaluate_acc(np.asarray(test['label']), yt_test)))

train accuracy: 0.95875
test accuracy: 0.828


In [ ]:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # pre-trained model
    emotions = load_dataset("emotion")
    model = (AutoModelForSequenceClassification.from_pretrained("bhadresh-savani/bert-base-uncased-emotion", num_labels=6).to(device))
    tokenizer = AutoTokenizer.from_pretrained("bhadresh-savani/bert-base-uncased-emotion")

    def tokenize(batch):
        return tokenizer(batch["text"], padding=True, truncation=True)

    def evaluate_acc(predictions):
        yt = np.argmax(predictions.predictions, axis=-1)
        y = predictions.label_ids
        accuracy = np.sum(y == yt) / y.shape[0]
        return accuracy

    emotions_tokenized = emotions.map(tokenize, batched=True, batch_size=None)
    training_args = TrainingArguments("test-trainer")
    trainer = Trainer(
        model,
        training_args,
        train_dataset=emotions_tokenized["train"],
        eval_dataset=emotions_tokenized["validation"],
        tokenizer=tokenizer,
    )

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

(…)uncased-emotion/resolve/main/config.json:   0%|          | 0.00/935 [00:00<?, ?B/s]

...

(…)ased-emotion/resolve/main/tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

(…)ion/resolve/main/special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:
    # accuracy before fine-tuning
    print("accuracy before fine-tuning:")
    predictions = trainer.predict(emotions_tokenized["train"])
    print("train accuracy: " + str(evaluate_acc(predictions)))

    predictions = trainer.predict(emotions_tokenized["test"])
    print("test accuracy: " + str(evaluate_acc(predictions)))
    print()

    trainer.train()

    # accuracy after fine-tuning
    print("accuracy after fine-tuning:")
    predictions = trainer.predict(emotions_tokenized["train"])
    print("train accuracy: " + str(evaluate_acc(predictions)))

    predictions = trainer.predict(emotions_tokenized["test"])
    print("test accuracy: " + str(evaluate_acc(predictions)))
    print()

    del model

    model = (AutoModelForSequenceClassification.from_pretrained("bhadresh-savani/bert-base-uncased-emotion", num_labels=6).to(device))
    training_args = TrainingArguments("test-trainer")
    trainer = Trainer(
        model,
        training_args,
        train_dataset=emotions_tokenized["train"],
        eval_dataset=emotions_tokenized["validation"],
        tokenizer=tokenizer,
    )

    for param in model.bert.encoder.parameters():
        param.requires_grad = False

    trainer.train()

    # accuracy after fine-tuning while freezing attetion weights
    print("accuracy after fine-tuning while freezing weights:")
    predictions = trainer.predict(emotions_tokenized["train"])
    print("train accuracy: " + str(evaluate_acc(predictions)))

    predictions = trainer.predict(emotions_tokenized["test"])
    print("test accuracy: " + str(evaluate_acc(predictions)))

Map:   0%|          | 0/16000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


accuracy before fine-tuning:


...

test accuracy: 0.9335



Step,Training Loss
500,0.060100
1000,0.064100
1500,0.073300
2000,0.071900
2500,0.037300
3000,0.048300
3500,0.044600
4000,0.056100
4500,0.029600
5000,0.046000


accuracy after fine-tuning while freezing weights:


train accuracy: 0.994


test accuracy: 0.929


In [ ]:
def extract_predictions(predictions, dataset):

    predicted_labels = np.argmax(predictions.predictions, axis=-1)
    true_labels = predictions.label_ids

    correct_predictions = []
    incorrect_predictions = []


    for i in range(len(true_labels)):

        if predicted_labels[i] == true_labels[i]:
            correct_predictions.append((dataset[i]['text'], true_labels[i], predicted_labels[i]))
        else:
            incorrect_predictions.append((dataset[i]['text'], true_labels[i], predicted_labels[i]))

    return correct_predictions, incorrect_predictions

In [ ]:
predictions_test = trainer.predict(emotions_tokenized["test"])
correct_preds_test, incorrect_preds_test = extract_predictions(predictions_test, emotions_tokenized["test"])

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
from bertviz import model_view, head_view
from transformers import AutoModel

model = AutoModel.from_pretrained("bhadresh-savani/bert-base-uncased-emotion", output_attentions=True)

tokenizer = AutoTokenizer.from_pretrained("bhadresh-savani/bert-base-uncased-emotion")
classes = ['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']

In [ ]:
def visualize_attention(inputs):
    viz_inputs = tokenizer.encode(inputs, return_tensors='pt')
    attention = model(viz_inputs)[-1]
    tokens = tokenizer.convert_ids_to_tokens(viz_inputs[0])
    model_view(attention, tokens, include_layers=[9],
        include_heads=[0])

In [ ]:
correct_preds_test[372][0] + ":  true label: " + str(classes[correct_preds_test[372][1]]) + " predicted label: " + str(classes[correct_preds_test[372][2]])

'ive been feeling afraid a lot lately:  true label: fear predicted label: fear'

In [ ]:
visualize_attention(correct_preds_test[372][0])

<IPython.core.display.Javascript object>

In [ ]:
correct_preds_test[222][0] + ":  true label: " + str(classes[correct_preds_test[222][1]]) + " predicted label: " + str(classes[correct_preds_test[222][2]])

'i was ambushed again it was apparently my fault again i feel worthless:  true label: sadness predicted label: sadness'

In [ ]:
visualize_attention(correct_preds_test[222][0])

<IPython.core.display.Javascript object>

In [ ]:
incorrect_preds_test[19][0] + ":  true label: " + str(classes[incorrect_preds_test[19][1]]) + " predicted label: " + str(classes[incorrect_preds_test[19][2]])

'i figured my parents wont make me feel accepted so i stopped trying i turned to romantic relationships with men:  true label: love predicted label: joy'

In [ ]:
visualize_attention(incorrect_preds_test[19][0])

<IPython.core.display.Javascript object>

In [ ]:
incorrect_preds_test[111][0] + ":  true label: " + str(classes[incorrect_preds_test[111][1]]) + " predicted label: " + str(classes[incorrect_preds_test[111][2]])

'i feel helpless to regain a safe feeling:  true label: sadness predicted label: fear'

In [ ]:
visualize_attention(incorrect_preds_test[111][0])

<IPython.core.display.Javascript object>